## Install lib

In [1]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

In [3]:
!rm -rf apex

In [ ]:
%%bash
git clone https://github.com/qywu/apex
cd apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
!pip install triton

In [ ]:
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed

In [7]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [YES] ...... [OKAY]
cpu_adagrad ............ [NO] ....... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
sparse_attn ............ [YES] ...... [OKAY]
transformer ............ [NO] ....... [OKAY]
stochastic_transformer . [NO] ....... [OKAY]
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not fou

Test, that deepspeed was installed correctly.

In [14]:
import deepspeed.ops.sparse_attention.sparse_attn_op

In [9]:
!rm -rf ru-gpts

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

In [ ]:
!pip install transformers==3.5.1

In [ ]:
!pip install timm==0.3.2

In [13]:
!cp ru-gpts/src_utils/trainer_pt_utils.py /usr/local/lib/python3.8/dist-packages/transformers/trainer_pt_utils.py

In [ ]:
!cp ru-gpts/src_utils/_amp_state.py /usr/local/lib/python3.8/dist-packages/apex/amp/_amp_state.py

Here restart colab env!!!

# Test model

In [3]:
import deepspeed

### Load model

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import sys
sys.path.append("ru-gpts/")

In [6]:
import os
os.environ["USE_DEEPSPEED"] = "1"

In [7]:
from src.xl_wrapper import RuGPT3XL

Note! seq_len is max sequence length for generation used in generation process. Max avialable seq_len is 2048 (in tokens).
Also inference takes around 10 Gb GPU memory.

In [8]:
import torch
torch.cuda.is_available()

True

In [9]:
from src import mpu

In [10]:
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "5000"

In [11]:
# gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)
gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)
# model parallel group is not initialized - если не подключена gpu

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
Use alternating sparse & dense attention layers


### Get logits

In [12]:
logits = gpt("Кто был президентом США в 2020? ").logits

In [13]:
type(logits), logits.shape

(torch.Tensor, torch.Size([1, 8, 50264]))

### Get loss

In [14]:
input_ids = [gpt.tokenizer("Кто был президентом США в 2020? ")['input_ids']]
labels = input_ids

In [15]:
import torch


with torch.no_grad():
    loss = gpt(input_ids=input_ids, labels=labels).loss

In [16]:
loss

[tensor(4.3895, device='cuda:0')]

### Simple generation

In [17]:
def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

Greedy decoding

In [18]:
filter_resuls(gpt.generate(
    "Кто был президентом США в 2020? ",
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Кто был президентом США в 2020? \nВ этом году выборы президента Соединенных Штатов Америки пройдут уже через несколько дней. И, как и всегда на протяжении последних лет (а это более чем 20-ти), кандидаты будут бороться за право стать главой государств']

sample

In [19]:
filter_resuls(gpt.generate(
    "Кто был президентом США в 2020? ", do_sample=True, num_return_sequences=5,
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Кто был президентом США в 2020? \nНовым, на первый взгляд странным и непонятным образом определилось будущее Америки. За несколько часов до конца таймера ожидания выборов президента уже ясно было - кто займет президентский пост после окончания голосования – Дональд Трам',
 'Кто был президентом США в 2020? \n"Я никогда не выйду живым из этого леса",- говорит главный герой фильма "Зеленая миля". Услышав это, начинаешь задумываться: а так ли хорош твой опыт жизни на другой стороне земного шара',
 'Кто был президентом США в 2020? \nВы хотите знать, как будет выглядеть ваш мир через сорок лет после того срока своего президентства уходящего следующего президента? Посмотрите на эти четыре изображения и попробуйте угадать кто бы это мог быть. \n Вот первый предполаг',
 'Кто был президентом США в 2020? \nВ интернете появился новый опрос на тему, которую так трудно назвать "демократичной". Народ хочет знать о том. какие президенты стояли у руля их страны и за сколько часов до собственной смерт

### Top_k top_p filtering

In [20]:
filter_resuls(gpt.generate(
    "Александр Сергеевич Пушкин родился в ",
    top_k=5,
    top_p=0.95,
    temperature=1.2,
    num_return_sequences=5,
    do_sample=True,
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Александр Сергеевич Пушкин родился в \nМоскве. В 1799 г., после того как его отец, отставной поручик лейб-гвардии Преображенского полка Александр Иванович (17451816), женился на вдове капитана Екатерине Петро',
 'Александр Сергеевич Пушкин родился в \xa01899 году. Он был сыном великого поэта Александра Сергеевича Пушкина и Натальи Гончаровой - сестры прапрабабушки Льва Толстого Александры Николаевны, которая была замужем за князем Николаем Васильевичем Трубецки',
 'Александр Сергеевич Пушкин родился в \n1817 году. Его отец – Александр Иванович, служил чиновником при министерстве внутренних дел Российской империи и умер рано; его мать Мария Алексеевна Ганнибал (урожденная Пушкина), урождённая Энгельга',
 'Александр Сергеевич Пушкин родился в \n1817 г. (по другим сведениям 1820). В детстве и ранней юности жил с родителями за границей, учился у лучших педагогов Франции - Лагарпа, Жозефа Мари Ашара. С ранних лет увле',
 'Александр Сергеевич Пушкин родился в \n1799 г. Его отец, Александр 

### Beamsearch

In [21]:
filter_resuls(gpt.generate(
    text="Александр Сергеевич Пушкин родился в ",
    max_length=50,
    num_beams=10,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
    num_return_sequences=5,
))

['Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать, урожденная Ганнибал, был',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать поэта, Мария Алексеевна Ганнибал',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать поэта, Наталья Николаевна \nПушкин',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать, урождённая Ганнибал, был',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \n